In [1]:
import requests 
from collections import defaultdict

In [2]:
docs_url = 'https://raw.githubusercontent.com/AdairPonceuwu/ch_llm/main/documents.json'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [3]:
documents[0]

{'text': 'El Eje 4 contribuirá al repoblamiento y mejoramiento urbano del Centro Histórico de Puebla, protegiendo su patrimonio histórico cultural.',
 'section': 'Vinculación a otros instrumentos municipales',
 'question': '¿Cómo contribuirá el Eje 4 a la protección del patrimonio histórico cultural del Centro Histórico?',
 'topic': 'PMD'}

In [4]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['topic']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [6]:
documents[3]

{'text': 'En el último decenio, la zona metropolitana registró su menor crecimiento poblacional con una tasa media anual de 0.9%. Esta situación contrasta con las altas tasas de crecimiento de municipios como Cuautlancingo, San Andrés Cholula, Huejotzingo y Coronango.',
 'section': 'Densidad poblacional',
 'question': '¿Cuál fue la tasa media anual de crecimiento poblacional en la zona metropolitana en el último decenio?',
 'topic': 'PMD',
 'id': 'bbcc69fc'}

In [7]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [8]:
len(hashes), len(documents)

(186, 186)

In [9]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [10]:
hashes['c78f3aa6']

[{'text': 'C6 Reservas ecológica, parques, jardines, espacio públcio y área verde',
  'section': 'Intervención Arquitectónica ',
  'question': '¿Qué establece la categoría 6 en el Programa Parcial de Desarrollo Urbano Sustentable?',
  'topic': 'PPDUS',
  'id': 'c78f3aa6'}]

In [11]:
import json

In [25]:
with open('documents-with-ids.json', 'w', encoding='utf-8') as f_out:
    json.dump(documents, f_out, ensure_ascii=False, indent=2)

In [258]:
prompt_template = """
Eres un excelente hablante de español y experto formulador de preguntas.
Formula 2 preguntas basadas en un registro de preguntas frecuentes (FAQ). Las preguntas que formules deben poder ser contestadas 
solamente con la respuesta de cada registro y deben ser completas y detalladas. 
Usa la menor cantidad posible de palabras exactas del registro.

Registro:

sección: {section}
pregunta: {question}
respuesta: {text}

No des texto extra, solo las preguntas formuladas, que deben de ir en el siguiente formato para que sea "parsable con JSON" sin utilizar "code blocks":

[
  "pregunta1",
  "pregunta2"
]


""".strip()

In [27]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [38]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='llama3.1',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [276]:
doc = documents[19]
questions = generate_questions(doc)

In [267]:
documents[19]

{'text': 'En la junta auxiliar de San Andrés Azumiatla, el 54.8% de las viviendas cuenta con apenas un dormitorio.',
 'section': 'Bienestar Social',
 'question': '¿Cuál es el porcentaje de viviendas con apenas un dormitorio en San Andrés Azumiatla?',
 'topic': 'PMD',
 'id': '318ff118'}

In [277]:
questions

'[\n  "¿Cuánto por ciento de viviendas tiene un solo cuarto en San Andrés Azumiatla?",\n  "En dónde se encuentra el 54.8% de las viviendas que son para dormir una sola en San Andrés Azumiatla?"\n]'

In [278]:
json.loads(questions)

['¿Cuánto por ciento de viviendas tiene un solo cuarto en San Andrés Azumiatla?',
 'En dónde se encuentra el 54.8% de las viviendas que son para dormir una sola en San Andrés Azumiatla?']

In [279]:
results["318ff118"] = questions

In [52]:
from tqdm.auto import tqdm

D:\Anaconda\envs\centro_historico\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
results = {}

In [178]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

100%|████████████████████████████████████████████████████████████████████████████████| 186/186 [53:45<00:00, 17.34s/it]


In [218]:
#results

In [12]:
import pickle
import json

In [14]:
#with open('results.bin', 'wb') as file:
#    pickle.dump(results, file)

In [13]:
with open('results.bin', 'rb') as file:
    results = pickle.load(file)

In [15]:
results

{'335b2f94': '[\n  "¿Qué acción ayudará al Eje 4 a proteger los elementos históricos del Centro Histórico?",\n  "En qué medida el Eje 4 contribuirá al mejoramiento urbano y preservación cultural de este área?"\n]',
 '805017f9': '[\n  "¿Cuáles son los objetivos principales del Plan de Manejo del Centro Histórico de la ciudad?",\n  "¿Qué acciones se llevarán a cabo para lograr un mayor compromiso con la gestión y conservación del patrimonio histórico?"\n]',
 '017273c8': '[\n  "¿Qué política abordan los programas parciales de desarrollo urbano del Centro Histórico?",\n  "¿Cuáles son los objetivos de gobierno en el Barrio de Santiago Universitario?"\n]',
 'bbcc69fc': '[\n  "¿Cuál fue la tasa más baja alcanzada por la zona metropolitana en cuanto a crecimiento poblacional durante el último decenio?",\n  "¿Cuáles son algunos de los municipios con las mayores tasas de crecimiento dentro del área metropolitana?"\n]',
 '9f8a1086': '[\n  "¿Qué fenómeno ha provocado la reducción de personas vivie

In [16]:
parsed_results = {}
successful_parses = 0
failed_parses = []

for doc_id, json_questions in results.items():
    try:
        parsed_results[doc_id] = json.loads(json_questions)
        successful_parses += 1
    except json.JSONDecodeError:
        failed_parses.append(doc_id)

In [17]:
successful_parses

186

In [18]:
doc_index = {d['id']: d for d in documents}

In [21]:
final_results = []

for doc_id, questions in parsed_results.items():
    topic = doc_index[doc_id]['topic']
    for q in questions:
        final_results.append((q, topic, doc_id))

In [22]:
import pandas as pd

In [27]:
df = pd.DataFrame(final_results, columns=['question', 'topic', 'document'])

In [28]:
df.to_csv('ground-truth-data.csv', index=False)

In [29]:
df.head()

,question,topic,document
0,¿Qué acción ayudará al Eje 4 a proteger los el...,PMD,335b2f94
1,En qué medida el Eje 4 contribuirá al mejorami...,PMD,335b2f94
2,¿Cuáles son los objetivos principales del Plan...,PMD,805017f9
3,¿Qué acciones se llevarán a cabo para lograr u...,PMD,805017f9
4,¿Qué política abordan los programas parciales ...,PMD,017273c8


In [30]:
!head ground-truth-data.csv

question,topic,document
¿Qué acción ayudará al Eje 4 a proteger los elementos históricos del Centro Histórico?,PMD,335b2f94
En qué medida el Eje 4 contribuirá al mejoramiento urbano y preservación cultural de este área?,PMD,335b2f94
¿Cuáles son los objetivos principales del Plan de Manejo del Centro Histórico de la ciudad?,PMD,805017f9
¿Qué acciones se llevarán a cabo para lograr un mayor compromiso con la gestión y conservación del patrimonio histórico?,PMD,805017f9
¿Qué política abordan los programas parciales de desarrollo urbano del Centro Histórico?,PMD,017273c8
¿Cuáles son los objetivos de gobierno en el Barrio de Santiago Universitario?,PMD,017273c8
¿Cuál fue la tasa más baja alcanzada por la zona metropolitana en cuanto a crecimiento poblacional durante el último decenio?,PMD,bbcc69fc
¿Cuáles son algunos de los municipios con las mayores tasas de crecimiento dentro del área metropolitana?,PMD,bbcc69fc
¿Qué fenómeno ha provocado la reducción de personas viviendo en el área princ